### 作業目的: 熟練以Torchtext進行文本資料讀取

本次作業主要會使用[polarity](http://www.cs.cornell.edu/people/pabo/movie-review-data/)的電影評論來進行使用torchtext資料讀取，學員可以在附件的polarity.tsv看到所使用的資料。

Hint: 這次作業同學可以嘗試使用[torchtext.data.TabularDataset](https://torchtext.readthedocs.io/en/latest/data.html#tabulardataset)，可以更簡易讀取資料

### 載入套件

In [1]:
import torch
import pandas as pd
import numpy as np
from torchtext import data, datasets
import spacy
spacy.load('en')

In [2]:
# 探索資料
# 可以發現資料為文本與類別，而類別即為正評與負評
input_data = pd.read_csv('./polarity.tsv', delimiter='\t', header=None, names=['text', 'label'])
input_data.head()

,text,label
0,films adapted from comic books have had plenty...,1
1,every now and then a movie comes along from a ...,1
2,you've got mail works alot better than it dese...,1
3,jaws is a rare film that grabs your attentio...,1
4,moviemaking is a lot like being the general ma...,1


### 建立Pipeline生成資料

In [3]:
# 建立Field與Dataset
text_field = data.Field(sequential=True, dtype=torch.float64, lower=True, tokenize='spacy')
label_field = data.Field(sequential=False)

In [4]:
# 取的examples並打亂順序
examples = []
for (text, label) in input_data.values:
    examples.append(data.Example.fromlist(data=[text, label],
                                          fields=[('text', text_field),
                                                  ('label', label_field)]))
np.random.shuffle(examples)

# 以8:2的比例切分examples
train_ex = examples[:int(len(examples)*0.8)]
test_ex = examples[int(len(examples)*0.8):]

# 建立training與testing dataset
train_data = data.Dataset(examples=train_ex, fields={'text':text_field, 'label':label_field})
test_data = data.Dataset(examples=test_ex, fields={'text':text_field, 'label':label_field})

train_data[0].label, train_data[0].text

(0,
 ['there',
  "'re",
  'so',
  'many',
  'things',
  'to',
  'criticize',
  'about',
  'i',
  'do',
  "n't",
  'know',
  'where',
  'to',
  'start',
  '.recommendation',
  ':',
  'turn',
  'off',
  'your',
  'brain',
  '-',
  'do',
  "n't",
  'be',
  'like',
  'me',
  ',',
  'decreasing',
  'the',
  'rating',
  'everyday',
  'because',
  'i',
  'think',
  'about',
  'it',
  'too',
  'much',
  '.a',
  'comet',
  'is',
  'about',
  'to',
  'strike',
  'earth',
  ',',
  'causing',
  'a',
  'catastrophe',
  'similar',
  'to',
  'the',
  'extinction',
  'level',
  'event',
  '(',
  'e',
  '.',
  'l',
  '.',
  'e',
  '.',
  ')',
  'that',
  'wiped',
  'out',
  'the',
  'dinosaurs',
  '.what',
  'follows',
  'is',
  'the',
  'story',
  'of',
  'a',
  'president',
  "'s",
  'bid',
  'to',
  'think',
  'for',
  'the',
  'good',
  'of',
  'his',
  'people',
  ',',
  'a',
  'rising',
  'reporter',
  ',',
  'the',
  'love',
  'story',
  'of',
  'two',
  'teenagers',
  '(',
  'one',
  'of',
  'w

In [5]:
# 建立字典
text_field.build_vocab(train_data)
label_field.build_vocab(train_data)

print(f"Vocabularies of index 0-5: {text_field.vocab.itos[:10]} \n")
print(f"words to index {text_field.vocab.stoi}")

Vocabularies of index 0-5: ['<unk>', '<pad>', ',', 'the', 'a', 'and', 'of', 'to', 'is', 'in'] 



words to index defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x000002622EE97C88>>, {'<unk>': 0, '<pad>': 1, ',': 2, 'the': 3, 'a': 4, 'and': 5, 'of': 6, 'to': 7, 'is': 8, 'in': 9, "'s": 10, '"': 11, 'that': 12, '-': 13, 'it': 14, ')': 15, '(': 16, '.': 17, 'with': 18, 'as': 19, 'film': 20, 'for': 21, 'his': 22, 'this': 23, '.the': 24, 'he': 25, 'on': 26, 'are': 27, 'but': 28, 'i': 29, 'be': 30, 'by': 31, "n't": 32, 'movie': 33, 'who': 34, 'an': 35, 'not': 36, 'one': 37, 'was': 38, 'have': 39, 'you': 40, 'has': 41, 'from': 42, 'at': 43, 'her': 44, 'they': 45, 'all': 46, '?': 47, 'out': 48, 'like': 49, 'about': 50, 'up': 51, 'more': 52, 'so': 53, 'or': 54, 'which': 55, 'their': 56, 'do': 57, ':': 58, 'what': 59, 'some': 60, 'does': 61, 'just': 62, '.it': 63, 'into': 64, 'there': 65, 'him': 66, 'she': 67, 'when': 68, "'": 69, 'than': 70, 'good': 71, 'time': 72, 'can': 73, 'only': 74, 'no': 75, 'would': 76, 'even': 77, 'we': 78, 'its': 79, 'story': 

In [6]:
# create iterator for training and testing data
train_iter = data.Iterator(train_data, batch_size=3, repeat=False, sort_key=lambda x: len(x.text))
test_iter = data.Iterator(test_data, batch_size=3, repeat=False, sort_key=lambda x: len(x.text))

In [7]:
for train_batch in train_iter:
    print(train_batch.text, train_batch.text.shape)
    print(train_batch.label, train_batch.label.shape)
    break

tensor([[4.0000e+00, 6.8000e+01, 2.7920e+03],
        [7.6400e+02, 1.4000e+01, 5.8000e+01],
        [1.3000e+01, 2.1300e+02, 3.0000e+00],
        ...,
        [1.0000e+00, 1.4000e+01, 1.0000e+00],
        [1.0000e+00, 8.0000e+00, 1.0000e+00],
        [1.0000e+00, 4.7000e+01, 1.0000e+00]], dtype=torch.float64) torch.Size([664, 3])
tensor([1, 1, 2]) torch.Size([3])
